# Getting LLC4320 output near the Maud rise area in the southern ocean

Want a smallish area with full water column data for u, v, SSH, salinity and potential temperature.
Want about 3 months of data at the hourly resolution

### First relevant imports

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import xarray as xr
import intake
import numpy as np
import xgcm
from xmitgcm import llcreader

In [2]:
import dask
# import xesmf as xe

### Using a dask cluster

Skip this for now

In [3]:
# from dask.distributed import LocalCluster
# from dask.distributed import Client

# cluster = LocalCluster()
# # cluster.scale(8)  # uses max of fig, but I think that is the default for locals
# # cluster.adapt(minimum=10, maximum=20)  # gives up to the same capacity as server fig in cloud

# client = Client(cluster)
# cluster

## Using xmitgcm llcreader to extract data

This will create a xarray dataset.

We directly request the time period we are interested in via the iter_start and _stop options.

We try to select chunk options that make this more efficient.

We also load all the grid information and transform to grid in lat/lon so we can easily subset our area/region later.

In [4]:
model = llcreader.ECCOPortalLLC4320Model()
dm = model.get_dataset(varnames=['Eta', 'U', 'V', 'Theta', 'Salt'], k_levels=range(0, 90),
                       k_chunksize=10, iter_start=273024,
                       iter_stop=587520, read_grid=True, type='latlon')  # getting surface, 3 months at hrly

#### This is how the dataset looks

In [5]:
dm

<xarray.Dataset>
Dimensions:  (face: 13, i: 17280, i_g: 17280, j: 12960, j_g: 12960, k: 90, k_l: 90, k_p1: 91, k_u: 90, time: 2184)
Coordinates:
  * k_p1     (k_p1) int64 0 1 2 3 4 5 6 7 8 9 ... 81 82 83 84 85 86 87 88 89 90
  * j        (j) int64 0 1 2 3 4 5 6 ... 12954 12955 12956 12957 12958 12959
  * k_u      (k_u) int64 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * i        (i) int64 0 1 2 3 4 5 6 ... 17274 17275 17276 17277 17278 17279
  * j_g      (j_g) int64 0 1 2 3 4 5 6 ... 12954 12955 12956 12957 12958 12959
  * time     (time) datetime64[ns] 2011-11-28 ... 2012-02-26T23:00:00
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * k_l      (k_l) int64 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * i_g      (i_g) int64 0 1 2 3 4 5 6 ... 17274 17275 17276 17277 17278 17279
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    CS       (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    SN       (j, i)

### Subset the region

using isel method after figuring out the i, j indices of the Maud rise area

In [6]:
# dm = dm.isel(i=slice(1650, 2330), j=slice(3190, 3780), i_g=slice(1650, 2330), j_g=slice(3190, 3780),)
dm = dm.isel(i=slice(1650+165, 2330-115), j=slice(3190+190, 3780-145), i_g=slice(1650+165, 2330-115), j_g=slice(3190+190, 3780-145),)

#### This is how much data we are going to be working with (in GB)

In [7]:
print("Full data set is ", dm.nbytes / 1e9)

print("One variable is ", dm['U'].nbytes/1e9)

321.794938196
80.19648


#### We start small so we also subset it in time

In [12]:
dm = dm.isel(time=slice(0, -1, 24))  # subset to daily
# dm = dm.isel(time=slice(0, -1, 24*30))  # subset monthly
# dm = dm.isel(time=0)  # one time instance

#### And we see the size is much smaller

In [14]:
print(dm.nbytes/1e9)
print(dm['U'].nbytes/1e9)

0.265216732
0.03672


#### A quick look at the surface salinity (note how long it takes to finish the plot)

In [13]:
%%time
plt.figure()
dm['Salt'].isel(time=0, k=0).plot()
# dm['Salt'].isel(k=0).plot()

ValueError: buffer size must be a multiple of element size

<Figure size 432x288 with 0 Axes>

### Now we try to get as much data as we can

But before that we will load them into memory; this will trigger all computations and transfer the data to our local memory space.

This is a good indication of how long it takes to actually have this data on your local machine.

First, we try with just one variable, $u$

In [12]:
with dask.config.set(scheduler='single-threaded'):
    with dask.diagnostics.ProgressBar():
        u = dm['U'].load()
#         u = dask.compute(dm['U'], retries=10)

[########################################] | 100% Completed | 12min 40.2s


ValueError: replacement data must match the Variable's shape

#### Now the full data set 

See the size estimate, and note the time it takes.

This larger data set may suffer from the intermittency of the NASA data server.

So if it is failing we will try something else below.

In [13]:
print(dm.nbytes/1e9)

with dask.config.set(scheduler='single-threaded'):
    with dask.diagnostics.ProgressBar():
        dsl = dm.load()
#         dsl = dask.compute(dm, retries=50)

0.265216732
[########################################] | 100% Completed | 48min 36.7s


ValueError: replacement data must match the Variable's shape

#### In case the above is failing, we can try this approach

The retries option in the dask compute method sometimes helps.

See this post: https://github.com/MITgcm/xmitgcm/issues/210#issue-641376849

In [14]:
with dask.config.set(scheduler='single-threaded'):
    with dask.diagnostics.ProgressBar():
#         u = dm['U'].load()
        u = dask.compute(dm['U'], retries=10)

[####################                    ] | 52% Completed |  3min 48.1s


ValueError: buffer size must be a multiple of element size

In [15]:
with dask.config.set(scheduler='single-threaded'):
    with dask.diagnostics.ProgressBar():
#         dsl = dm.load()
        dsl = dask.compute(dm, retries=50)

[############################            ] | 71% Completed | 14min 16.4s


ValueError: buffer size must be a multiple of element size